In [1]:
import numpy as np
from tqdm import tqdm
import cv2 as cv
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as T
from torch.utils.data import Dataset, random_split, DataLoader
import torch.nn.functional as F

import json
import os
import random
from collections import OrderedDict

In [2]:
track_id2student_id_it = {7:0, 6:1, 0:2, 8:3, 1:4, 3:5, 11:6, 15:7, 16:8, 2:9, 5:12, 9:13, 4:14} # irimi tenkan, my tracking algo
track_id2student_id_ta = {8:0, 7:1, 2:2, 10:3, 3:4, 4:5, 14:6, 9:7, 1:8, 12:9, 11:10, 0:11, 6:12, 5:13} # tsugi ashi, my tracking algo
track_id2student_id_utmt = {7:0, 6:1, 1:2, 8:3, 3:4, 5:5, 11:6, 10:7, 2:8, 0:11, 4:13, 9:9, 12:10, 19:12} # utmt, my tracking algo

single_student_tids = {"1156":0, "1237":1, "0756":0, "0912":0, "1043":0}

In [3]:
file2mapping = {"../alphapose_res/irimi_tenkan_kps.json":track_id2student_id_it,
                "../alphapose_res/ta_tracking.json":track_id2student_id_ta,
                "../alphapose_res/utmt_kps.json":track_id2student_id_utmt}
annotations_map = {"../alphapose_res/irimi_tenkan_kps.json":"../data/irimi_tenkan/annotations.json",
                "../alphapose_res/ta_tracking.json":"../data/tsugi_ashi/annotations.json",
                "../alphapose_res/utmt_kps.json":"../data/ushiro_tenkan_mae_tenkan/annotations.json"}
file2exercise = {"../alphapose_res/irimi_tenkan_kps.json":"ирими тенкан", 
                 "../alphapose_res/ta_tracking.json":"цуги аши",
                 "../alphapose_res/utmt_kps.json":"уширо тенкан мае тенкан"}
single_st_ann = "../data/single_student_annotations.json"

SEQ_LEN_FRAMES = 512
CHANNELS = 3

# Подготовка данных

In [4]:
from dataproc import *
from train_utils import * 

In [5]:
ss_vid_ann = [os.path.join("../alphapose_res", f) for f in ["0756_alpha.json", "0912_alpha.json",
                                                            "1043_alpha.json", "1156_alpha.json", 
                                                            "1237_alpha.json"]]
student_exercise_points, exercise_borders = parse_borders_annotations(file2mapping, single_st_ann, 
                                                                      annotations_map, ss_vid_ann, 
                                                                      single_student_tids)

In [96]:
frame_durs = [stex.shape[1] for stex in student_exercise_points]
encoded_borders = encode_borders(exercise_borders, frame_durs, "<EXERCISE_BODY>")
split_exercs, split_borders = split_into_equal_frame_len(student_exercise_points, encoded_borders, SEQ_LEN_FRAMES, drop_last=False)